# Project Benson - Zip Code Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon
import matplotlib.path as mplPath
import re
import json, urllib3
import shapefile
from string import punctuation
import pickle

%matplotlib inline

#Set options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 3)

#### Read in NYC Zip Code shapefile

In [ ]:
def save_geojson(url= 'http://catalog.civicdashboards.com/dataset/11fd957a-8885-42ef-aa49-5c879ec93fac/resource/28377e88-8a50-428f-807c-40ba1f09159b/download/nyc-zip-code-tabulation-areas-polygons.geojson'):
    response = urllib3.PoolManager().request('GET', url)
    with open('NYC_zipcode.txt', 'w') as f:
        json.dump(json.loads(response.data), f)
        
def read_geojson(json_file='/Users/samfunk/ds/metis/metis-one-Benson/NYC_zipcode.txt'):
    with open(json_file, 'r') as f:
        return json.load(f)

zip_json = read_geojson()

#### Convert shapfile to dictionary

In [ ]:
def zip_dictionary(zip_json=zip_json):
    zip_dict = {}
    for i in zip_json['features']:
        zip_dict[int(i['properties']['postalCode'])] = i['geometry']['coordinates'][0]
    return zip_dict

zip_dict = zip_dictionary()

def zipPath(zip_dict=zip_dict):
    '''
    Copy original zip code dictionary and convert coordinates to Path values for searching
    '''
    zip_path = zip_dict.copy()
    for key, value in zip_path.items():
        zip_path[key] = mplPath.Path(value)
    return zip_path

zip_path = zipPath()

#### Read in MTA station data

In [ ]:
def read_shapefile(mta= '/Users/samfunk/Downloads/MTA_STATIONS/geo_export_34e54a93-5cf1-4a42-a24f-8e2108d208cb.shp'):
    return shapefile.Reader(mta)

mta_shp = read_shapefile()

### Format stations and add zip code to each

In [ ]:
def stations_zip(mta_shp=mta_shp):
    '''
    Using shapefile package, convert .shp to readable/usable dictionary
    '''
    stations = []
    for feature in mta_shp.shapeRecords():
        coords = feature.shape.__geo_interface__['coordinates']
        stop = feature.record[2]
        line = feature.record[-1:][0]
        s = [stop, line, coords]
        stations.append(s)
    return stations
stations = stations_zip()


def find(stations, zips):
    '''
    Loop through each station to figure out which zip code it belongs to
    ---
    IN: stations dictionary, zip code path dictionary
    OUT: updated stations dictionary with appended zip codes
    '''
    for station_ind, station_stop in enumerate(stations):
        coord = station_stop[2]
        for zip_key, zip_value in zips.items():
            if zip_value.contains_point(coord):
                stations[station_ind].append(zip_key)
    return stations
station_info = find(stations, zip_path)


def clean_lines(stations=station_info):
    '''
    Drop 'Express' from line and give missing zip codes to stations
    Return clean dataframe
    '''
    for ind, val in enumerate(stations):
        raw = val[1].split('-')
        for route in raw:
            if 'Express' in route:
                raw.remove(route)
        stations[ind][1] = ''.join(raw)

    missing = ['231st St','238th St','Bowling Green','Broad Channel','Carroll St','South Ferry','Van Cortlandt Park - 242nd St','Whitehall St']
    no_zip = [10463,10463,10004,11693,11231,10004,10471,10004]
    i = 0
    for ind, j in enumerate(stations):
        if missing[i] == j[0]:
            stations[ind].append(no_zip[i])
            i += 1

        if j[0] == '125th St' and j[1] == '456':
            stations[ind].append(10035)

    #replace_125 = {'125th St': {'line': '456', zipCode=10035}}
    return station_info
station_info = clean_lines()

#### Convert to pandas dataframe

In [ ]:
columns = ['station', 'linename', 'coordinates', 'zipcode']
df = pd.DataFrame(station_info, columns=columns)

## Clean station and line names to match MTA data

In [ ]:
def clean_stations(name):
    '''
    Use regex to make multiple changes to station names, specifically remove 'st', 'nd', 'rd', and 'th' from strings.
    Hard coding for special edged cases
    '''
    r = re.compile(r'[{}]+'.format(re.escape(punctuation)))
    stop_split = r.split(name.upper())

    d = []
    for j, y in enumerate(stop_split):
        if re.search(r'^\s', y):
            y = re.sub(r'^\s', '', y)

        if re.search(r'\s$', y):
            y = re.sub(r'\s$', '', y)

        if j < 2:
            if re.search(r'\d', y):
                if 'BEACH' in y:
                    regex = re.search(r'(BEACH\s\d+)\w*(\s\w+)', y)
                    d.append(regex[1] + regex[2])
                elif 'BAY' in y:
                    d.append('BAY 50 ST')
                elif re.search(r'^[EW]\s', y):
                    regex = re.search(r'([EW]\s\d*)\w*(\s\w*)', y)
                    d.append(regex[1] + regex[2])
                elif re.search(r'(\d*)\w*(\s\w*)', y):
                    regex = re.search(r'\s?(\d*)\w*(\s\w*)', y)
                    d.append(regex[1] + regex[2])
                elif re.search(r'^\d*\w', y):
                    regex = re.search(r'(\d+)\w+', y)
                    d.append(regex[1])
            elif 'TREMONT' in y:
                continue
            elif re.search(r'^[A-Z]{2}\s', y):
                regex = re.search(r'(^[A-Z]{2}\s[A-Z]+)', y)
                d.append(regex[1])
            elif re.search(r'^BAY\s', y) or re.search(r'^AVE\s', y) or re.search(r'^VAN\s', y) or re.search(r'^NEW\s', y):
                regex = re.search(r'(^[A-Z]+\s[A-Z]*)', y)
                d.append(regex[1])
            elif 'GRAND' in y:
                d.append(y)
            elif re.search(r'\s', y):
                regex = re.search(r'([A-Z]+)\s([A-Z]{2})', y)
                d.append(regex[1] + ' ' + regex[2])
            else:
                d.append(y)
        elif j == 2 and 'ROCKEFELLER' in y:
            d.append('ROCK')

    return '-'.join(d)
df['station'] = df['station'].apply(clean_stations)

df['station'] = df['station'].str.replace(' AVE', ' AV')
df['station'] = df['station'].str.replace('AVE ', 'AVENUE ')
df['station'] = df['station'].str.replace(' PKY', ' PKWY')

def clean_lines(name):
    if name == '46':
        return '6'
    else:
        return name
df['linename'] = df['linename'].apply(clean_lines)

df['station_linename'] = df['station'] + ' ' + df['linename']


#### Read in pickle files of cleaned MTA dataframe

These dataframes are sorted by morning and evening rush hours. They have average entires and exits per hour along with the census data containing the median income and estimate for young women with STEM degrees both by zip code. All four of these variables have their own repsective percentile ranks, which will come into play below.

In [ ]:
def read_pickle(filename):
    with open('/Users/samfunk/ds/metis/metis-project-one/' + filename, 'rb') as pf:
        return pickle.load(pf)

morning = read_pickle('morning_rush.pkl')
evening = read_pickle('evening_rush.pkl')

## Calculate Station Score for morning and evening rush hours

For this section, two scores are calculated for each station, one for the morning rush hours and one for the evevning ones. Using the percentiles from the imported dataframe, we will apply weights for each stations' values. The weights used for each parameter may be adjusted given the organization's preferences. For now, we decided to assign both station traffic and young women in STEM a 0.4 and income a 0.2. This allotment allows traffic and women both to be doubly important when compared to income, which tends to crowd out the other variables given a larger weight in a city like New York.

In [ ]:
def score(df, weights, time):
    '''
    IN: df = imported pickle file dataframes, weights = list of weights (must sum to 1), time = dummy string for function conditional
    OUT: new df column 'score'
    '''
    station_weight = weights[0]
    income_weight = weights[1]
    stem_weight = weights[2]
    
    df.rename(columns={'Average Entries Per Hour, Percentile': 'entries_rank', 'Average Exits Per Hour, Percentile': 'exits_rank', 'Median income (dollars); Estimate; Households, Percentile': 'median_income_rank', 'Demographic, Percentile': 'stem_rank'})
    
    if time == 'morning':
        df['score'] = station_weight*df['entries_rank'] + income_weight*df['median_income_rank'] + stem_weight*df['stem_rank']
    else:
        df['score'] = station_weight*df['exits_rank'] + income_weight*df['median_income_rank'] + stem_weight*df['stem_rank']

score(morning, [0.4, 0.2, 0.4], 'morning')
score(evening, [0.4, 0.2, 0.4], 'evening')



## Plot stations and zip codes together

In [ ]:
plot_morning = morning.sort_values('score', ascending=False)[['station', 'linename', 'coordinates', 'zip code', 'score']].dropna(axis=0)
plot_evening = evening.sort_values('score', ascending=False)[['station', 'linename', 'coordinates', 'zip code', 'score']].dropna(axis=0)

def plot_map(zip_dict, df):
    '''
    Plot the zip code boundaries and MTA stations on a single plot, where the size of the station points are dependent on their scores
    IN: zip_dict = zip code dictionary from above, df = morning or evening rush hour dataframe
    OUT: plot of NYC zip codes and subway stations
    '''
    fig,ax = plt.subplots()
    ax.set_aspect('equal')
    
    #Plot station coordinates
    lon = []
    lat = []
    s = [10*size for size in df['test score']]
    for coords in df['coordinates']:
        lon.append(coords[0])
        lat.append(coords[1])
    plt.scatter(lon, lat, s=s, c='black')
    
    #Plot zip code boundaries
    for key, value in list(zip_dict.items()):
        x_lon = np.zeros((len(value),1))
        y_lat = np.zeros((len(value),1))
        for ip in range(len(value)):
            x_lon[ip] = value[ip][0]
            y_lat[ip] = value[ip][1]

        plt.plot(x_lon, y_lat, linewidth=1)


    WEST_LONGITUDE = -74.05
    EAST_LONGITUDE = -73.83
    NORTH_LATITUDE = 40.92
    SOUTH_LATITUDE = 40.65

    plt.xlim(WEST_LONGITUDE, EAST_LONGITUDE)
    plt.ylim(SOUTH_LATITUDE, NORTH_LATITUDE)
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

In [ ]:
plot_map(zip_dict, plot_morning)
plt.title('MTA stations scores (Morning Rush)')

In [ ]:
plot_map(zip_dict, plot_evening)
plt.title('MTA stations scores (Evening Rush)')